In [35]:
#for windows users:
#pip install bitermplus

#for mac users:
# xcode-select --install
# brew install libomp
# pip3 install bitermplus

# pip install tmplot
# pip install tomotopy

In [36]:
import bitermplus as btm
import numpy as np
import pandas as pd

import json 
import csv
import nltk
import unicodedata
import re
import sys
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
import gensim, spacy
from gensim.models.ldamulticore import LdaMulticore
import re
from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [37]:
df = pd.read_csv("../pos_neg_neu_india.csv")
df

,Unnamed: 0,index,Text,new_text,languages,translated,new_languages,pos_tagged_words,words_processed_all,words_processed_noun_adj_verb_adv,words_processed_noun,text_type,sentiment
0,0,0,soniya mummy hoti toh covid me magic stick ghu...,soniya mummy hoti toh covid me magic stick ghu...,Unknown,soniya mummy hoti toh covid me magic stick ghu...,Unknown,"[('soniya', 'JJ'), ('mummy', 'NN'), ('hot', 'J...","['soniya', 'mummy', 'hot', 'toh', 'couid', 'ma...",['death'],['death'],subjective,0
1,11,11,خود کو اور اپنے پیاروں کو کورونا وبا سے بچانے ...,خود کو اور اپنے پیاروں کو کورونا وبا سے بچانے ...,"URDU, Unknown",Vaccinate to protect yourself and your loved o...,ENGLISH,"[('vaccinate', 'NN'), ('to', 'TO'), ('protect'...","['vaccinate', 'protect', 'loved_one', 'coronav...","['uranus', 'covidvaccination']",['covidvaccination'],subjective,1
2,12,12,is covid ki waja se buht saray logon ne apne p...,is covid ki waja se buht saray logon ne apne p...,Unknown,is covid ki waja se buht saray logon ne apne p...,Unknown,"[('is', 'VBZ'), ('couid', 'JJ'), ('ki', 'NN'),...","['couid', 'ki', 'raja', 'se', 'spray', 'logon'...","['hair', 'ply', 'vaccination']","['hair', 'ply', 'vaccination']",subjective,2
3,18,18,deaths due covid what is vaccination rate what...,deaths due covid what is vaccination rate what...,"ENGLISH, Unknown",deaths due covid what is vaccination rate what...,"ENGLISH, Unknown","[('deaths', 'NNS'), ('due', 'JJ'), ('couid', '...","['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'couid', 'vaccination', 'rate', 'har...",subjective,2
4,28,28,vaccination against covid- 19 is necessity tod...,vaccination against covid- 19 is necessity tod...,ENGLISH,vaccination against covid- 19 is necessity tod...,ENGLISH,"[('vaccination', 'NN'), ('against', 'IN'), ('c...","['vaccination', 'couid', 'nineteen', 'necessit...","['necessity', 'today', 'scenario', 'hope', 'pr...","['necessity', 'today', 'scenario', 'problem']",subjective,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43458,95179,95179,nature seems have already started working savi...,nature seems have already started working savi...,ENGLISH,nature seems have already started working savi...,ENGLISH,"[('nature', 'NN'), ('seems', 'VBZ'), ('have', ...","['nature', 'seem', 'already', 'start', 'work',...","['nature', 'seem', 'already', 'start', 'work',...","['nature', 'generation', 'vaccinesaveslive', '...",subjective,2
43459,95180,95180,"dear , we #india 🇮🇳has helped this #world incl...","dear , we #india Indiahas helped this #world i...",ENGLISH,"dear , we #india Indiahas helped this #world i...",ENGLISH,"[('dear', 'IN'), ('we', 'PRP'), ('india', 'VBP...","['dear', 'india', 'indians', 'help', 'world', ...","['dear', 'help', 'world', 'include', 'kindness...","['world', 'kindnessmatter', 'vaccinesaveslive']",subjective,2
43460,95184,95184,our sincere request state central govt 🙏🙏. ple...,our sincere request state central govt folded_...,ENGLISH,our sincere request state central govt folded_...,ENGLISH,"[('our', 'PRP$'), ('sincere', 'JJ'), ('request...","['sincere', 'request', 'state', 'central', 'go...","['sincere', 'request', 'state', 'central', 'go...","['request', 'state', 'govt', 'share', 'vaccine...",subjective,2
43461,95185,95185,thanks your appreciation valuable feedback abo...,thanks your appreciation valuable feedback abo...,ENGLISH,thanks your appreciation valuable feedback abo...,ENGLISH,"[('thanks', 'NNS'), ('your', 'PRP$'), ('apprec...","['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'feedback', 'vaccine...",subjective,2


In [38]:
df.drop(columns = ['Unnamed: 0', 'Text', 'index', 'new_text', 'languages', 'new_languages', 'pos_tagged_words'], inplace = True)
df

,translated,words_processed_all,words_processed_noun_adj_verb_adv,words_processed_noun,text_type,sentiment
0,soniya mummy hoti toh covid me magic stick ghu...,"['soniya', 'mummy', 'hot', 'toh', 'couid', 'ma...",['death'],['death'],subjective,0
1,Vaccinate to protect yourself and your loved o...,"['vaccinate', 'protect', 'loved_one', 'coronav...","['uranus', 'covidvaccination']",['covidvaccination'],subjective,1
2,is covid ki waja se buht saray logon ne apne p...,"['couid', 'ki', 'raja', 'se', 'spray', 'logon'...","['hair', 'ply', 'vaccination']","['hair', 'ply', 'vaccination']",subjective,2
3,deaths due covid what is vaccination rate what...,"['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'couid', 'vaccination', 'rate', 'har...",subjective,2
4,vaccination against covid- 19 is necessity tod...,"['vaccination', 'couid', 'nineteen', 'necessit...","['necessity', 'today', 'scenario', 'hope', 'pr...","['necessity', 'today', 'scenario', 'problem']",subjective,2
...,...,...,...,...,...,...
43458,nature seems have already started working savi...,"['nature', 'seem', 'already', 'start', 'work',...","['nature', 'seem', 'already', 'start', 'work',...","['nature', 'generation', 'vaccinesaveslive', '...",subjective,2
43459,"dear , we #india Indiahas helped this #world i...","['dear', 'india', 'indians', 'help', 'world', ...","['dear', 'help', 'world', 'include', 'kindness...","['world', 'kindnessmatter', 'vaccinesaveslive']",subjective,2
43460,our sincere request state central govt folded_...,"['sincere', 'request', 'state', 'central', 'go...","['sincere', 'request', 'state', 'central', 'go...","['request', 'state', 'govt', 'share', 'vaccine...",subjective,2
43461,thanks your appreciation valuable feedback abo...,"['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'feedback', 'vaccine...",subjective,2


In [39]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [40]:
tokens_words_processed_noun = list(sent_to_words(df['words_processed_noun']))

In [41]:
from nltk.corpus import stopwords
stop_list = stopwords.words('english')
high_freq_known_words = ['vaccine', 'vaccination', 'covidvaccine', 'vaccinate', 'vaccinated', 'amp', 'couid', 
             'covishield', 'people', 'country', 'pfizer', 'hour', 'year', 'month', 'time', 'day', 'covid',
             'yesterday', 'thing', 'sir', 'today', 'retweet', 'corona', 'one', 'love', 'coronavirus', 
              'ahmedabad', 'person', 'week', 'other', 'today', 'use', 'part', 'covidvaccination', 
             'thank', 'crore', 'thing', 'twentynine', 'need', 'lot']

stop_list.extend(high_freq_known_words)

test_words_only1 = ['not', 'vaccine', 'covid', 'and', 'vegan', 'cruelty', 'free', 'i', 'use', 'elf', 'cosmetics', 'every', 'day', 'and', 'i', 'love', 'it']
test_stopremoved = [w for w in test_words_only1 if w not in stop_list or w == 'not']
print(test_stopremoved)

['not', 'vegan', 'cruelty', 'free', 'elf', 'cosmetics', 'every']


In [42]:
stop_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [43]:
df['words_processed_noun'] = tokens_words_processed_noun

In [44]:
# remove stop words from tokenised_words_only

tokenised_stopremoved = [[w for w in each if w not in stop_list or w == 'not'] for each in tokens_words_processed_noun]
tokenised_stopremoved

[['death'],
 [],
 ['hair', 'ply'],
 ['death', 'rate', 'harm', 'harm'],
 ['necessity', 'scenario', 'problem'],
 ['population',
  'yrs',
  'offline',
  'class',
  'examination',
  'situation',
  'safety',
  'class',
  'examination'],
 ['way', 'millennium', 'peril', 'job', 'process'],
 ['symptom', 'example', 'number', 'death', 'population', 'life'],
 ['follower', 'pledge', 'service', 'humanity'],
 ['case', 'case', 'testing', 'taboo'],
 ['intellectual',
  'education',
  'leader',
  'health',
  'system',
  'pandemic',
  'nation',
  'service'],
 ['memory', 'post', 'post', 'forgetting', 'memory', 'couple'],
 ['agreement', 'student', 'travel', 'guideline'],
 ['system',
  'conversion',
  'appeasement',
  'health',
  'service',
  'infection',
  'rate',
  'death',
  'rate',
  'rate',
  'welfare',
  'isis'],
 ['personnel', 'unitedstate'],
 ['oppression', 'appointment', 'wife', 'worker', 'present', 'money'],
 ['oppression', 'appointment', 'wife', 'center', 'worker'],
 ['result',
  'effect',
  'exer

In [45]:
df['words_processed_noun'].values[0][0]

'death'

In [46]:
df['tokenised_stopremoved'] = tokenised_stopremoved

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43463 entries, 0 to 43462
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   translated                         43463 non-null  object
 1   words_processed_all                43463 non-null  object
 2   words_processed_noun_adj_verb_adv  43463 non-null  object
 3   words_processed_noun               43463 non-null  object
 4   text_type                          43463 non-null  object
 5   sentiment                          43463 non-null  int64 
 6   tokenised_stopremoved              43463 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.3+ MB


In [48]:
df['sentiment'].value_counts()

0    20029
2    17594
1     5840
Name: sentiment, dtype: int64

In [49]:
# positive_df = df[df['sentiment'] == 2]

In [50]:
# positive_df

In [51]:
texts = df['words_processed_noun']

In [52]:
new_text_noun = []

for row in texts:
    # print(type(row))
    new_row = " ".join(row)
    new_text_noun.append(new_row)


In [53]:
new_text_noun

['death',
 '',
 'hair ply vaccination',
 'death couid vaccination rate harm harm',
 'necessity today scenario problem',
 'population yrs offline class amp examination couid situation safety class amp examination couid vaccination',
 'way thing millennium peril couid job vaccination process retweet',
 'symptom vaccination example country number death population couid life',
 'follower pledge couid vaccination service humanity',
 'case case testing people vaccination taboo',
 'intellectual education leader health system pandemic nation service',
 'people memory post post vaccination forgetting memory couple year',
 'agreement couid student travel guideline',
 'system conversion appeasement health service people infection rate death rate vaccination rate welfare people isis',
 'personnel unitedstate',
 'oppression people yesterday appointment couid vaccine wife today vaccination worker present time money',
 'oppression people yesterday appointment couid vaccine wife today center vaccinati

In [54]:
df['new_text_noun'] = new_text_noun

In [55]:
texts = df['new_text_noun']

In [56]:
texts

0                                                death
1                                                     
2                                 hair ply vaccination
3               death couid vaccination rate harm harm
4                     necessity today scenario problem
                             ...                      
43458                nature generation life strengthen
43459                             world kindnessmatter
43460                         request state govt share
43461                      thank appreciation feedback
43462    dose covidvaccine amp amp follow vaccine folk
Name: new_text_noun, Length: 43463, dtype: object

In [89]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

high_freq_known_words = ['vaccine', 'vaccination', 'covidvaccine', 'vaccinate', 'vaccinated', 'amp', 'couid', 
             'covishield', 'people', 'country', 'pfizer', 'hour', 'year', 'month', 'time', 'day', 'covid',
             'yesterday', 'thing', 'sir', 'today', 'retweet', 'corona', 'one', 'love', 'coronavirus', 
              'ahmedabad', 'person', 'week', 'other', 'today', 'use', 'part', 'covidvaccination', 
             'thank', 'crore', 'thing', 'twentynine', 'need', 'lot', 'number', 'life']

stop_words.extend(high_freq_known_words)

In [58]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

# X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
# docs_vec = btm.get_vectorized_docs(texts, vocabulary)
# biterms = btm.get_biterms(docs_vec)

# # Initializing and running model
# model = btm.BTM(
#     X, vocabulary, seed=12321, T=4, M=20, alpha=50/4, beta=0.09)
# model.fit(biterms, iterations=600)

In [59]:
# stable_topics = [0, 1, 2, 3]
# top_words = btm.get_top_topic_words(
#     model,
#     words_num=12,
#     topics_idx=stable_topics)

In [60]:
# top_words

## 5 topics

In [61]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

# X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
# docs_vec = btm.get_vectorized_docs(texts, vocabulary)
# biterms = btm.get_biterms(docs_vec)

# # Initializing and running model
# model = btm.BTM(
#     X, vocabulary, seed=12321, T=5, M=20, alpha=50/5, beta=0.09)
# model.fit(biterms, iterations=600)

In [62]:
# stable_topics = [0, 1, 2, 3, 4]
# top_words = btm.get_top_topic_words(
#     model,
#     words_num=12,
#     topics_idx=stable_topics)

In [63]:
# top_words

## 6 topics

In [64]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=6, M=20, alpha=50/6, beta=0.09)
model.fit(biterms, iterations=600)

100%|█████████████████████████████████████████| 600/600 [00:14<00:00, 40.71it/s]


In [65]:
stable_topics = [0, 1, 2, 3, 4, 5]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [66]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5
0,virus,student,dose,government,mask,injection
1,infection,school,hospital,state,family,fever
2,case,child,center,world,life,pain
3,death,travel,certificate,govt,virus,shot
4,risk,test,staff,population,hand,booster
5,booster,kid,slot,money,world,body
6,dose,case,help,price,friend,symptom
7,shot,exam,appointment,nation,home,needle
8,immunity,government,family,citizen,team,dose
9,number,life,government,life,fight,arm


## 7 topics

In [67]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=7, M=20, alpha=50/7, beta=0.09)
model.fit(biterms, iterations=600)

100%|█████████████████████████████████████████| 600/600 [00:16<00:00, 35.84it/s]


In [68]:
stable_topics = [0, 1, 2, 3, 4, 5, 6]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [69]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6
0,virus,student,dose,mask,government,video,fever
1,infection,school,hospital,family,state,family,injection
2,case,child,center,virus,world,man,pain
3,death,travel,certificate,life,govt,woman,body
4,booster,test,slot,health,population,mom,needle
5,risk,kid,staff,team,money,friend,booster
6,shot,exam,appointment,dose,price,dog,symptom
7,dose,case,help,work,nation,home,shot
8,immunity,government,parent,way,supply,cat,side_effect
9,number,teacher,number,worker,citizen,life,arm


## 8 topics

In [90]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=8, M=20, alpha=50/8, beta=0.09)
model.fit(biterms, iterations=600)

100%|█████████████████████████████████████████| 600/600 [00:16<00:00, 36.49it/s]


In [91]:
p_zd8 = model.transform(docs_vec)

100%|█████████████████████████████████| 43463/43463 [00:00<00:00, 216230.52it/s]


In [92]:
stable_topics = [0, 1, 2, 3, 4, 5, 6, 7]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [93]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7
0,virus,world,dose,student,family,government,mask,fever
1,infection,election,hospital,school,health,state,hand,injection
2,case,death,center,child,world,govt,distance,booster
3,death,man,certificate,travel,team,price,family,shot
4,risk,leader,slot,kid,worker,population,virus,pain
5,dose,party,staff,test,work,hospital,place,body
6,booster,photo,appointment,exam,virus,money,home,symptom
7,immunity,nation,help,case,help,world,case,needle
8,shot,medium,family,teacher,care,supply,face,dose
9,patient,issue,parent,government,drive,company,crowd,friend


## 9 topics

In [94]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=9, M=20, alpha=50/9, beta=0.09)
model.fit(biterms, iterations=600)

100%|█████████████████████████████████████████| 600/600 [00:18<00:00, 33.15it/s]


In [95]:
stable_topics = [0, 1, 2, 3, 4, 5, 6, 7, 8]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [96]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8
0,virus,man,government,student,travel,mask,staff,fever,dose
1,infection,world,state,school,test,virus,health,injection,hospital
2,case,death,world,child,quarantine,family,family,pain,certificate
3,death,woman,govt,kid,home,hand,worker,body,slot
4,booster,photo,population,exam,rule,fight,hospital,needle,center
5,shot,video,price,teacher,flight,way,center,symptom,appointment
6,risk,leader,hospital,parent,mask,dose,work,doctor,parent
7,dose,news,money,case,state,world,team,booster,help
8,immunity,map,citizen,class,government,friend,care,shot,coin
9,patient,election,supply,college,mall,distance,employee,arm,mother


## 10 topics

In [97]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=10, M=20, alpha=50/10, beta=0.09)
model.fit(biterms, iterations=600)

100%|█████████████████████████████████████████| 600/600 [00:18<00:00, 32.56it/s]


In [98]:
stable_topics = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [99]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,student,world,staff,virus,travel,fever,government,dose,video,mask
1,school,population,family,infection,test,injection,price,hospital,post,hand
2,child,state,health,case,quarantine,booster,hospital,certificate,doctor,virus
3,kid,government,work,death,home,pain,money,slot,hair,distance
4,exam,govt,worker,risk,rule,shot,state,center,map,family
5,teacher,election,center,booster,flight,body,company,appointment,news,home
6,parent,leader,team,immunity,state,symptom,govt,parent,case,syringe
7,case,nation,drive,dose,mask,needle,supply,help,fire,friend
8,class,death,hospital,shot,government,dose,world,coin,story,face
9,college,wave,government,patient,mall,doctor,production,mother,world,woman


## Get topic distribution

In [80]:
matrix8 = pd.DataFrame(data=p_zd8,columns=['Topic0','Topic1','Topic2','Topic3','Topic4','Topic5','Topic6','Topic7'])

In [81]:
dominant_topic8 = []
for each in p_zd8:
    topic = np.argmax(each)
    dominant_topic8.append(topic)

In [82]:
matrix8['dominant'] = dominant_topic8

In [83]:
sum_up_to_one8 = []

for each in p_zd8:
    if sum(each) == 0:
        result = "Unclassified"
    else:
        result = "Classified"
    sum_up_to_one8.append(result)

In [84]:
matrix8['sum_up'] = sum_up_to_one8

In [85]:
matrix8[matrix8['sum_up']=='Unclassified']

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,dominant,sum_up
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Unclassified
48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Unclassified
66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Unclassified
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Unclassified
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Unclassified
...,...,...,...,...,...,...,...,...,...,...
43429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Unclassified
43431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Unclassified
43448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Unclassified
43451,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Unclassified


In [88]:
matrix8.groupby('dominant').count()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,sum_up
dominant,,,,,,,,,
0,5981,5981,5981,5981,5981,5981,5981,5981,5981
1,3209,3209,3209,3209,3209,3209,3209,3209,3209
2,7590,7590,7590,7590,7590,7590,7590,7590,7590
3,5573,5573,5573,5573,5573,5573,5573,5573,5573
4,3269,3269,3269,3269,3269,3269,3269,3269,3269
5,5385,5385,5385,5385,5385,5385,5385,5385,5385
6,3111,3111,3111,3111,3111,3111,3111,3111,3111
7,9345,9345,9345,9345,9345,9345,9345,9345,9345


In [ ]:
matrix8.to_csv('check_matrix8.csv')

In [ ]:
#43463